<a href="https://colab.research.google.com/github/doyeo-n/-Capstone-8_code/blob/main/%5B8%5DConvLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#필요패키지 다운
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from impyute.imputation.cs import mice

!pip install dask
!pip install "dask[dataframe]"
!pip install livelossplot
!pip install keijzer
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import numpy as np
from tqdm import tqdm
import dask.dataframe as dd

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Conv1D, MaxPool2D, Flatten, Dropout, CuDNNGRU, CuDNNLSTM
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from time import time
from tensorflow.keras.layers import BatchNormalization
#from keras.layers.normalization import BatchNormalization
from livelossplot import PlotLossesKeras
from keras.layers.advanced_activations import LeakyReLU, PReLU
import tensorflow as tf
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
#from keras.utils.training_utils import multi_gpu_model
from tensorflow.python.client import device_lib
from sklearn.preprocessing import StandardScaler
from keras.layers import InputLayer, ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization
#from keijzer import *

%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"} # Make sure the axis background of plots is white, this is usefull for the black theme in JupyterLab
sns.set()

In [ ]:
def df_to_lstm_format(df, test_size=0.3, look_back=5, target_column='target', scale_X=True):
    """
    TODO: output train and test datetime
    Input is a Pandas DataFrame. 
    Output is a np array in the format of (samples, timesteps, features).
    Currently this function only accepts one target variable.
    Usage example:
    # variables
    df = data # should be a pandas dataframe
    test_size = 0.5 # percentage to use for training
    target_column = 'c' # target column name, all other columns are taken as features
    scale_X = False
    look_back = 5 # Amount of previous X values to look at when predicting the current y value
    """
    from array import array
    df = df.set_index(index_col)
    sha = df.shape
    a = round(sha[0]*0.3)
    b = len(df[:-a])
    c = b%7
    d = len(df[-a : -6])
    e = d%7
    print(a, b, c, d, e)
    train, test = df[c:-a], df[-a+e:-6] 
    print(train.shape, test.shape)
    train =  np.array(train)
    test = np.array(test)

    # ...train
    X_train, y_train = train[:,:-1], train[:,-1]
    X_test, y_test = test[:,:-1], test[:,-1]
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    # Scale the features

    if scale_X:
        scalerX = StandardScaler(with_mean=True, with_std=True).fit(X_train)
        X_train = scalerX.transform(X_train)
        X_test = scalerX.transform(X_test)
  
    # Reshape the arrays
    target_location = 9
    samples = len(X_train) # in this case 217 samples in the training set
    num_features = target_location # All columns before the target column are features

    samples_train = X_train.shape[0] - look_back
    X_train_reshaped = np.zeros((samples_train, look_back, num_features))
    y_train_reshaped = np.zeros((samples_train))

    for i in range(samples_train):
        y_position = i + look_back
        X_train_reshaped[i] = X_train[i:y_position]
        y_train_reshaped[i] = y_train[y_position]


    samples_test = X_test.shape[0] - look_back
    X_test_reshaped = np.zeros((samples_test, look_back, num_features))
    y_test_reshaped = np.zeros((samples_test))

    for i in range(samples_test):
        y_position = i + look_back
        X_test_reshaped[i] = X_test[i:y_position]
        y_test_reshaped[i] = y_test[y_position]
    
    return X_train_reshaped, y_train_reshaped, X_test_reshaped, y_test_reshaped


In [ ]:
def split(df):
  df = df.set_index(index_col)
  
  X_train, y_train, X_test, y_test = df_to_lstm_format(df=df, test_size=test_size, look_back=look_back, target_column='log_cyan', scale_X=True)
  X_train.shape

  X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, X_train.shape[2], 1)
  X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, X_test.shape[2], 1)
  X_train.shape
  return X_train, y_train, X_test, y_test


def mape(y_true, y_pred):
    import keras.backend as K
    """
    Returns the mean absolute percentage error.
    For examples on losses see:
    https://github.com/keras-team/keras/blob/master/keras/losses.py
    """
    return (K.abs(y_true - y_pred) / K.abs(y_pred)) * 100

def smape(y_true, y_pred):
    import keras.backend as K
    """
    Returns the Symmetric mean absolute percentage error.
    For examples on losses see:
    https://github.com/keras-team/keras/blob/master/keras/losses.py
    """
    return (K.abs(y_pred - y_true) / ((K.abs(y_true) + K.abs(y_pred))))*100

def mae(y_true, y_pred):
    import keras.backend as K

    return (K.abs(y_true - y_pred))
def build__model():

  # define LSTM model
  model = Sequential()

  # Input shape: (samples, time, channels, rows, cols) see: https://keras.io/layers/recurrent/#convlstm2d
  model.add(ConvLSTM2D(
          filters=8,
          kernel_size=(10, 10),
          input_shape=(7, 1, 9, 1),
          padding='same',
          return_sequences=True))
  model.add(LeakyReLU())
  model.add(BatchNormalization())

  model.add(ConvLSTM2D(
          filters=8,
          kernel_size=(5, 5),
          padding='same',
          return_sequences=False))
  model.add(LeakyReLU())
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Flatten())

  model.add(Dense(32, kernel_initializer='TruncatedNormal'))
  model.add(LeakyReLU())
  model.add(Dropout(0.2))

  model.add(Dense(16, kernel_initializer='TruncatedNormal'))
  model.add(LeakyReLU())
  model.add(Dropout(0.2))

  model.add(Dense(1))
  sgd = SGD(lr=0.00001, momentum=0.9, decay=0, nesterov=True) # sgd in general yields better results, but needs a lot of tweeking and is slower
  adam = Adam(lr=lr)

  # compile & fit
  model.compile(optimizer=adam, loss = ['mse'], metrics=[mape, smape, mae])

  model.summary()
  return model


def pred_result(df):
  def mse(targets, predictions):
      return ((predictions - targets) ** 2).mean()
  def rmse(targets, predictions):
      return np.sqrt(((predictions - targets) ** 2).mean())
  def rsquared(x, y):
      slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
      return r_value**2

  from sklearn.metrics import mean_absolute_error

  def mae(targets, predictions): 
    return mean_absolute_error(targets.reshape(-1, 1), predictions.reshape(-1, 1))
  magnitude = 1
  y_preds = model.predict(X_test)
  y_true = y_test.reshape(y_test.shape[0], 1)
  
  split_index = int(df.shape[0]*test_size)
  x = df[split_index:]
  #len(y_true), len(x)

  datetime_difference = len(x) - len(y_true)
  x = x[datetime_difference:] # Correct for datetime difference, this is a dirty way of doing it

  plt.figure(figsize=(20,10))
  plt.plot(x.index, y_true, '.-', color='red', label='Real values', alpha=0.5)
  plt.plot(x.index, y_preds, '.-', color='blue', label='Predicted values')

  plt.ylabel(r'gasPower $\cdot$ 10$^{-%s}$ [m$^3$/h]' % magnitude, fontsize=14)
  plt.xlabel('datetime [-]', fontsize=14) #TODO: set x values as actual dates

  plt.legend(loc='upper left', borderaxespad=0, frameon=False, fontsize=14, markerscale=3)

  mse_result = model.evaluate(X_test, y_test)[0]
  mape_result = model.evaluate(X_test, y_test)[1]
  smape_result = model.evaluate(X_test, y_test)[2]
  mae_result = model.evaluate(X_test, y_test)[3]
  from sklearn.metrics import mean_absolute_error

  rmse = mse_result**(1/2)
  mse_result = model.evaluate(X_test, y_test)[0]
  mape_result = model.evaluate(X_test, y_test)[1]
  smape_result = model.evaluate(X_test, y_test)[2]
  mse_result = mse(y_true, y_preds)
  #rmse = rmes(y_true, y_preds)
  #rsquared = rsquared(y_true, y_preds)


  plt.title('ConvLSTM2D result \n MSE = %.2f \n RMSE = %.3f \n MAPE = %.1f [%%] \n MAE = %.3f [%%] ' % (mse_result, rmse, mape_result, mae_result), fontsize = 14)

  #plt.savefig('figures/LSTM result hourly.png', dpi=300)
  print('FINISHED')

In [ ]:
#결정사항
look_back = timestep # D -> 5, H -> 5*24
num_features = len(input_list)
output_dim = len(output_col)
test_size = 0.7
hidden_nodes = 35 # 35
lr= 1e-4
epochs = 5 #135
bs = 2**6

#모델링
X_train, y_train, X_test, y_test = df_to_lstm_format(df=imputed_data, test_size=test_size, look_back=look_back, target_column=output_col, scale_X=True)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, X_test.shape[2], 1)

early_stopping_monitor = EarlyStopping(patience=5000)
model = build__model()
model.fit(X_train, y_train, epochs=epochs, batch_size=bs, validation_data=(X_test, y_test),verbose=1, callbacks=[PlotLossesKeras(), early_stopping_monitor])
pred_result(imputed_data)